# **DIC driven FE interpretation of a shear zone**

This notebook shows how to derive a deeper interpretation of the DIC displacements measured on a grid.
Based on an FE-mesh, it delivers the strain, stress and damage tensors. Further processing of the 
anisotropic damage tensor is used to detect the localized cracks.

In [1]:
%matplotlib widget

In [2]:
import ibvpy.api as ib
import matplotlib.pylab as plt
from mayavi import mlab
from bmcs_shear.api import CrackBridgeAdv
from bmcs_shear.dic_crack import \
    DICGrid, DICAlignedGrid, DICCOR, DICStateFields
from bmcs_shear.dic_crack.dic_state_fields import DICStateFields
from matplotlib import cm
import numpy as np
np.seterr(divide='ignore', invalid='ignore');

# DIC grid

## Input displacements on a grid

In [3]:
dic_grid = DICGrid(start_t=0, end_t=1, U_factor=100, dir_name='B1_TV1')

In [4]:
dic_grid = DICGrid(start_t = 0, end_t =26, L = 1500, U_factor = 1, dir_name='B1_TV2')

In [5]:
dic_grid = DICGrid(start_t=0, end_t=1, U_factor=100, dir_name='B10_TV2')

## Finite element processing of the displacement field

In [6]:
dsf = DICStateFields(dic_grid=dic_grid, t=0.5)
dsf.tmodel_.trait_set(E=5000, c_T=0, nu=0.18, epsilon_0=0.0005, epsilon_f=0.01);
dsf.eval()

# State fields on a regular grid

In [7]:
dsf.interact()

# TODO - in the `DICStateFields` class

 - decompose the fields into properties

# Next steps in crack detection

 - identify the crack tip 
 - identify the index regions between cracks in the final stage (reference frame -- and rotating frame)
 - improve the detection of the cracks at the bottom - avoid multiple cracks next to each other
 - incremental analysis - backward and forward crack processing
 - exploit the principle damage direction to limit the range of the crack increment
 - how to detect the start of the failure crack - this would only work for incremental evaluation?

# Detection provided 

Along the detected crack, process the crack opening and sliding starting from the tip.

With the fixed and rotating frames detected - process the COR histories 

Derive the moment - rotation relation for each crack

Use the rigid rotation kinematics to evaluate the opening and sliding - compare with the experimental values

Apply the values needed for comparison 

In [8]:
eps_EMab, eps_MNab, eps_MNa, max_eps_MN = dsf.eps_fields

In [9]:
x_MNa = dsf.x_MNa
x_MN, y_MN = np.einsum('...a->a...', x_MNa)

In [10]:
dsf.mlab_scalar(x_MN.T, y_MN.T, dsf.omega_MN.T, factor=100)

In [11]:
x_M, x_N = x_MN[:, 0], y_MN[0, :]
xx_M = np.linspace(x_M[-1], x_M[0], 100)
yy_N = np.linspace(x_N[0], x_N[-1], 30)
xx_NM, yy_NM = np.meshgrid(xx_M, yy_N)
u_NMa = dsf.interp_U(xx_M, yy_N)

TypeError: interp_U() takes 2 positional arguments but 3 were given

In [ ]:
from mayavi import mlab
from tvtk.api import tvtk
import numpy as np

def mlab_vector(x_NM, y_NM, vector_NMa, factor=100, label='displ'):
    mlab.figure()
    scene = mlab.get_engine().scenes[-1]
    scene.name = label
    scene.scene.background = (1.0, 1.0, 1.0)
    scene.scene.foreground = (0.0, 0.0, 0.0)
    scene.scene.z_plus_view()
    scene.scene.parallel_projection = True
    pts_shape = x_NM.shape + (1,)
    pts = np.empty(pts_shape + (3,), dtype=float)
    pts[..., 0] = x_NM[..., np.newaxis]
    pts[..., 1] = y_NM[..., np.newaxis]
    len_vector_NM = np.sqrt(np.einsum('...a,...a->...', vector_NMa, vector_NMa))
    pts[..., 2] = len_vector_NM[..., np.newaxis] * factor
    pts = pts.transpose(2, 1, 0, 3).copy()
    pts.shape = int(pts.size / 3), 3
    sg = tvtk.StructuredGrid(dimensions=pts_shape, points=pts)
    sg.point_data.scalars = len_vector_NM.ravel()
    sg.point_data.scalars.name = 'length'
#     delta_23 = np.array([[1, 0, 0], [0, 1, 0]], dtype=np.float_)
#     vector_NMa_3D = np.einsum('...a,ab->...b', vector_NMa, delta_23)
#     sg.point_data.vectors = vector_NMa_3D.reshape(-1, 3)
#     sg.point_data.vectors.name = label
    # Now visualize the data.
    d = mlab.pipeline.add_dataset(sg)
    mlab.pipeline.iso_surface(d)
    mlab.pipeline.surface(d)
    mlab.show()


In [ ]:
len_vector_NM = np.sqrt(np.einsum('...a,...a->...', u_NMa, u_NMa))
len_vector_NM.shape

In [ ]:
mlab_vector(xx_NM, yy_NM, u_NMa)